## **Spider**
---

En esta parte usamos la API que tiene CIMA para conseguir un fichero CSV para los distintos medicamentos que existen. Este fichero CSV se llamará `medicamentos.csv` y contendrá la siguiente información:
- `Nregistro`: Número de registro del medicamento.
- `Nombre`: Nombre del medicamento.
- `Principios_activos`: Nombres de los principios activos.
- `Pdf_url`: enlace al PDF de la ficha técnica del medicamento.

Este fichero CSV se guardará en la carpeta `data/outputs/1_data_acquisition/spyder/` y después lo tendremos que cargar en `data/inputs/1_data_acquisition/fetcher/` para poder usarlo en el siguiente paso (esto también se hace en este script).

In [1]:
import requests
import pandas as pd
import math
import os

In [ ]:
url = "https://cima.aemps.es/cima/rest/medicamentos"
headers = {"Accept": "application/json"}
tamanioPagina = 25

# Primera petición para conocer totalFilas
try:
    params = {
        "pagina": 1,
        "tamanioPagina": tamanioPagina
    }
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    totalFilas = data.get("totalFilas", 0)
except Exception as e:
    print("Error en la solicitud inicial:", e)
    totalFilas = 0

if totalFilas == 0:
    print("No se han obtenido resultados.")
    registros = []
else:
    total_paginas = math.ceil(totalFilas / tamanioPagina)
    print("Total de medicamentos:", totalFilas)
    print("Total de páginas:", total_paginas)

    registros = []
    for pagina in range(1, total_paginas + 1):
        params = {
            "pagina": pagina,
            "tamanioPagina": tamanioPagina
        }
        print(f"Recuperando página {pagina} de {total_paginas}...")
        try:
            response = requests.get(url, params=params, headers=headers)
            data = response.json()
        except Exception as e:
            print(f"Error en la página {pagina}: {e}")
            continue  # Salta a la siguiente página si hay error
        
        for med in data.get("resultados", []):
            try:
                nregistro = med.get("nregistro")
                nombre_med = med.get("nombre")
                # Extraer el nombre de los principios activos desde "vtm"
                principios_activos = med.get("vtm", {}).get("nombre")
                
                # Buscar en "docs" el documento de tipo 1 (ficha técnica en PDF)
                pdf_url = None
                for doc in med.get("docs", []):
                    if doc.get("tipo") == 1:
                        pdf_url = doc.get("url")
                        break

                registros.append({
                    "nregistro": nregistro,
                    "nombre": nombre_med,
                    "principios_activos": principios_activos,
                    "pdf_url": pdf_url
                })
            except Exception as e:
                print("Error procesando medicamento:", e)
    
df = pd.DataFrame(registros)

# Eliminar duplicados
# Ordenar df_resultados por el campo nregistro de forma descendente
df.sort_values("nregistro", ascending=False, inplace=True)
print(f"Nº registros en df_resultados: {len(df)}")

# Eliminar duplicados en el campo nombre, quedándonos con el primer registro (último registro temporalmente hablando)
df.drop_duplicates(subset="nombre", keep="first", inplace=True)
print(f"Nº registros tras eliminar duplicados: {len(df)}")

Total de medicamentos: 25076
Total de páginas: 1004
Recuperando página 1 de 1004...
pagina 1
Recuperando página 2 de 1004...
Recuperando página 3 de 1004...
Recuperando página 4 de 1004...
Recuperando página 5 de 1004...
Recuperando página 6 de 1004...
Recuperando página 7 de 1004...
Recuperando página 8 de 1004...
Recuperando página 9 de 1004...
Recuperando página 10 de 1004...
Recuperando página 11 de 1004...
Recuperando página 12 de 1004...
Recuperando página 13 de 1004...
Recuperando página 14 de 1004...
Recuperando página 15 de 1004...
Recuperando página 16 de 1004...
Recuperando página 17 de 1004...
Recuperando página 18 de 1004...
Recuperando página 19 de 1004...
Recuperando página 20 de 1004...
Recuperando página 21 de 1004...
Recuperando página 22 de 1004...
Recuperando página 23 de 1004...
Recuperando página 24 de 1004...
Recuperando página 25 de 1004...
Recuperando página 26 de 1004...
Recuperando página 27 de 1004...
Recuperando página 28 de 1004...
Recuperando página 29 de

In [ ]:
# Guardar el datafrane en un archivo CSV
df.to_csv(os.path.join("..", "..", "data", "outputs", "1_data_acquisition", "spider","Medicamentos.csv"), index=False)

In [ ]:
# Copio (guardo) este fichero Medicamentos.csv en la carpeta inputs/1_data_acquisition/fetcher para seguir ejecutando
df.to_csv(os.path.join("..", "..", "data", "inputs", "1_data_acquisition", "fetcher","Medicamentos.csv"), index=False)
